In [ ]:
import numpy as np
import pandas as pd
import time
import random
from helper_functions import *

In [ ]:
#problem_instance = "inst_tuning\heur040_n_300_m_13358"
instance_type = "test_instances/" #"inst_competition/" #"inst_tuning/" # 
problem_instance = "heur002_n_100_m_3274" #"heur040_n_300_m_13358" #"heur001_n_10_m_31" 
path = "data/"+instance_type + problem_instance +".txt"

In [ ]:
nodes, edges, s = create_problem_instance(path)
is_splex(nodes = nodes, plex_number = 2, s=s)

# Construction heuristic

In [ ]:
constr_nodes, constr_edges, constr_score = construction_heuristic(nodes,edges,s)
print("number of plexes: "+ str(len(constr_nodes["splex"].unique())))
print("score: " +str(constr_score))
print(constr_edges)

# Randomized Construction heuristic

In [ ]:
rand_nodes, rand_edges, rand_score = randomized_greedy(nodes, edges, s, alpha = 0.75)
print("number of plexes: "+ str(len(rand_nodes["splex"].unique())))
print("score: " +str(rand_score))

# extract 1 node neighborhood

In [ ]:
extract_nodes, extract_edges, extract_score = extract1node(constr_nodes, constr_edges, constr_score, s, step = "best")
print("number of plexes: "+ str(len(extract_nodes["splex"].unique())))
print("score: " +str(extract_score))

# merge 2 plexes neighborhood

In [ ]:
merged_nodes, merged_edges, merged_score = merge2plexes(constr_nodes, constr_edges, constr_score, s, 
                                                        step = "best", add_neg_edges = True)
print("number of plexes: "+ str(len(merged_nodes["splex"].unique())))
print("score: " +str(merged_score))


In [ ]:
merged_nodes2, merged_edges2, merged_score2 = merge2plexes(constr_nodes, constr_edges, constr_score, s, 
                                                        step = "best", add_neg_edges = False)
print("number of plexes: "+ str(len(merged_nodes2["splex"].unique())))
print("score: " +str(merged_score2))

## do it again on the new best solution

In [ ]:
merged_nodes3, merged_edges3, merged_score3 = merge2plexes(merged_nodes, merged_edges, merged_score, s, step = "best")
print("number of plexes: "+ str(len(merged_nodes2["splex"].unique())))
print("score: " +str(merged_score2))

# einmal alles (mit scharf)

In [ ]:
instance_type = "inst_competition/"#"test_instances/" #"inst_competition/" #"inst_tuning/" # 
problem_instance = "heur051_n_300_m_20122" #"heur040_n_300_m_13358" #"heur001_n_10_m_31" 
path = "data/"+instance_type + problem_instance +".txt"
nodes, edges, s = create_problem_instance(path)
print(edges)
constr_nodes, constr_edges, constr_score = construction_heuristic(nodes, edges, s)
write_solution(constr_edges, problem_instance, algorithm = "construction")
print(constr_edges)

In [ ]:
def vnd(solution: Solution, neighborhoods: list[Neighborhood]) -> Solution:
    current_best: Solution = solution
    best_objective: int = current_best.get_objective()
    
    i = 0 # First neighborhood
    while i < len(neighborhoods):
        current_neighborhood = neighborhoods

        best_in_neighborhood: int = None
        best_definition = None
        neighbor_definitions: list = current_neighborhood.get_all_neighbors()
        for definition in neighbor_definitions:
            current_objective: int = current_neighborhood.get_objective(current_best, definition)
            if (best_in_neighborhood == None) or (current_objective < best_in_neighborhood):
                best_in_neighborhood = current_objective
                best_definition = definition

        if best_in_neighborhood < best_objective:
            current_best = current_neighborhood.get_solution(current_best, best_definition)
            i = 0
        else:
            i += 1

    return current_best

def local_search(solution: Solution, neighborhood: Neighborhood):
    best_solution: Solution = solution
    best_objective: int = None

    neighbor_definitions: list = neighborhood.get_all_neighbors()
    for definition in neighbor_definitions:
        current_objective: int = neighborhood.get_objective(solution, definition)

        if (best_objective == None) or (current_objective < best_objective):
            best_objective = current_objective
            best_solution = neighborhood.get_solution(solution, definition)

    return best_solution

def grasp(problem: Problem, neighborhood: Neighborhood) -> Solution:
    best_solution: Solution = None
    for i in range(20):
        current_solution = problem.get_randomized_solution()
        local_optimum = local_search(current_solution, neighborhood)
        if (best_solution == None) or (local_optimum.get_objective() < best_solution.get_objective()):
            best_solution = local_optimum
    
    return best_solution



In [ ]:
from typing import Callable

problem = Problem(path)
first_improvement: Callable[[Solution], Solution] = lambda sol: merge2plexes(sol.get_nodes(), sol.get_edges(), sol.get_weight(), sol.get_s(), step="first")
best_improvement: Callable[[Solution], Solution] = lambda sol: merge2plexes(sol.get_nodes(), sol.get_edges(), sol.get_weight(), sol.get_s(), step="best")
random_improvement: Callable[[Solution], Solution] = lambda sol: merge2plexes(sol.get_nodes(), sol.get_edges(), sol.get_weight(), sol.get_s(), step="random")

neighborhood = Neighborhood(first_improvement, best_improvement, random_improvement)
inital_solution = problem.get_heuristic_solution()
local_best = neighborhood.get_best_improvement(inital_solution)
print(inital_solution.get_weight())
print(local_best.get_weight())